In [1]:
import json
import pandas as pd
from pprint import pprint
from typing import Any, Dict, List
import textwrap


def print_keys_formatted(keys: List[str], indent: int = 0) -> None:
    """Print keys in a formatted way, 10 per line."""
    keys_str = ", ".join(keys)
    wrapped = textwrap.wrap(
        keys_str, width=80, initial_indent=" " * indent, subsequent_indent=" " * indent)
    for line in wrapped:
        print(line)


def explore_value_structure(value: Any, indent: int = 0, parent_key: str = "") -> None:
    """Recursively explore and print structure of any value type."""
    indent_str = "│   " * (indent // 4) + "    " * (indent % 4)

    if isinstance(value, dict):
        print(f"{indent_str}└── {parent_key} (Dictionary)")
        print(f"{indent_str}    Number of keys: {len(value)}")
        if value:
            print(f"{indent_str}    Keys:")
            print_keys_formatted(list(value.keys()), indent + 8)
            print(f"{indent_str}    Structure of each key:")
            for key, sub_value in value.items():
                explore_value_structure(sub_value, indent + 8, key)
    elif isinstance(value, list):
        print(f"{indent_str}└── {parent_key} (List)")
        print(f"{indent_str}    Length: {len(value)}")
        if value:
            print(f"{indent_str}    First item structure:")
            explore_value_structure(value[0], indent + 8, "First Item")
            if len(value) > 1:
                print(
                    f"{indent_str}    Note: {len(value)-1} more items with similar structure")
    else:
        print(f"{indent_str}└── {parent_key}")
        print(f"{indent_str}    Type: {type(value).__name__}")
        if isinstance(value, (str, int, float, bool)):
            print(f"{indent_str}    Value: {value}")


def explore_json_structure(file_path: str) -> Dict:
    """Load and explore JSON file structure."""
    print("=" * 80)
    print(f"Exploring JSON file: {file_path}")
    print("=" * 80)

    # Load the JSON file
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Print the type of data and its length if it's a list
    print(f"\nRoot level:")
    explore_value_structure(data, 0, "Root")

    return data


if __name__ == "__main__":
    file_path = "appraisals_dataset.json"
    data = explore_json_structure(file_path)
    print(f"Number of appraisals: {len(data['appraisals'])}")
    print(type(data['appraisals']))
    print(f"Keys in the first appraisal: {data['appraisals'][0].keys()}")
    print(f"keys in second appraisal: {data['appraisals'][1].keys()}")
    print(f"************************************************")
    print(type(data['appraisals'][0]['subject']))
    print(
        f"Keys in the subject of the first appraisal: {list(data['appraisals'][0]['subject'].keys())}")
    print(f"************************************************")
    print(type(data['appraisals'][0]['comps']))
    print(
        f"Keys in the comps of the first appraisal: {list(data['appraisals'][0]['comps'][0].keys())}")
    print(f"************************************************")
    print(type(data['appraisals'][0]['properties']))
    print(
        f"Keys in the properties of the first appraisal: {list(data['appraisals'][0]['properties'][0].keys())}")

Exploring JSON file: appraisals_dataset.json

Root level:
└── Root (Dictionary)
    Number of keys: 1
    Keys:
        appraisals
    Structure of each key:
│   │   └── appraisals (List)
│   │       Length: 88
│   │       First item structure:
│   │   │   │   └── First Item (Dictionary)
│   │   │   │       Number of keys: 4
│   │   │   │       Keys:
                        orderID, subject, comps, properties
│   │   │   │       Structure of each key:
│   │   │   │   │   │   └── orderID
│   │   │   │   │   │       Type: str
│   │   │   │   │   │       Value: 4762597
│   │   │   │   │   │   └── subject (Dictionary)
│   │   │   │   │   │       Number of keys: 35
│   │   │   │   │   │       Keys:
                                address, subject_city_province_zip,
                                effective_date, municipality_district,
                                site_dimensions, lot_size_sf, units_sq_ft,
                                year_built, structure_type, roofing,
              

In [2]:
import polars as pl
subjects = []
for appriasal in data['appraisals']:
    subjects.append(appriasal['subject'])

print(len(subjects))

88


In [3]:
subjects_df = pl.from_dicts(subjects)
print(subjects_df.shape)
subjects_df.head(10)

(88, 35)


address,subject_city_province_zip,effective_date,municipality_district,site_dimensions,lot_size_sf,units_sq_ft,year_built,structure_type,roofing,effective_age,style,construction,remaining_economic_life,windows,basement,exterior_finish,basement_area,foundation_walls,flooring,plumbing_lines,heating,fuel_type,water_heater,cooling,room_count,num_beds,room_total,main_lvl_area,second_lvl_area,third_lvl_area,gla,subject_age,num_baths,condition
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""142-950 Oakview Ave Kingston O…","""""Twin Oak Meadows""""","""Apr/11/2025""","""Kingston""","""Condo Common Property""","""n/a""","""SqFt""","""1976""","""Townhouse""","""Asphalt Shingle""","""25""","""2 Storey""","""Wood Frame""","""50""","""Vinyl""","""Full/Finished""","""Brick Veneer, Vinyl Siding""","""522""","""Concrete""","""Hardwood, carpet, laminate, ce…","""Copper, PEX, ABS""","""Forced Air""","""Natural Gas""","""40 Gallon - Gas""","""Central Air""","""6""","""3""","""6""","""522""","""522""","""""","""1044""","""49""","""1:1""","""Average"""
"""7180 207 HWY Halifax NS B0J2L0""","""West Chezzetcook, NS B0J2L0""","""Apr/17/2025""","""Halifax Regional Municipality …","""See Schedule A and or Plot Map""","""72745+/-SqFt""","""SqFt""","""2011""","""Detached""","""Asphalt Shingle""","""10""","""1.5 Storey""","""Log""","""50""","""Wood""","""Full/Finished""","""Log""","""1060""","""Poured Concrete""","""Carpet""","""Copper, PEX, ABS""","""Radiant""","""Electric""","""80 +/- gl Electric""","""Ductless mini split""","""6""","""3""","""6""","""1060""","""440""","""""","""1500 SqFt""","""14+/-yrs""","""2:1""","""Average"""
"""11 PAUL AVE Ayr ON N0B1E0""","""Ayr ON N0B1E0""","""May/01/2025""","""Township of North Dumfries, Re…","""131' x 154'""","""20174 SqFt""","""SqFt""","""1983""","""Detached""","""Asphalt/Fiberglass Shingle""","""15""","""2 Storey""","""Wood""","""65""","""PVC""","""Full/Part Finished""","""Brick/Siding""","""1660""","""Concrete""","""Laminate""","""Copper, PEX, ABS""","""Forced Air""","""Natural Gas""","""Natural Gas""","""Central Air""","""9""","""4""","""9""","""1660""","""1340""","""""","""3000 SqFt""","""42""","""2:1""","""Average"""
"""102 Stonewalk Dr Kemptville ON…","""Kemptville ON K0G 1J0""","""Apr/15/2025""","""North Grenville, Ontario""","""176.38 F x Irregular""","""1.25 Acres""","""Acres""","""2012 +/-""","""Detached""","""Asphalt/Fiberglass Shingle""","""10""","""Bungalow""","""Wood""","""60""","""Vinyl""","""Full/Finished""","""Brick Veneer/Vinyl Siding""","""1283""","""Poured concrete""","""Hardwood""","""Copper, PEX, ABS""","""Forced Air""","""Natural Gas""","""Natural Gas""","""Central Air""","""5""","""2""","""5""","""""","""""","""""","""1283 SqFt""","""2012 +/-""","""3:0""","""Good"""
"""407 105 Dunbrack St Halifax NS…","""Halifax NS B3M3G7""","""Apr/17/2025""","""Halifax Regional Municipality …","""CONDO - N/A""","""N/A""","""N/A""","""1978""","""Condominium""","""Tar & Gravel""","""15""","""1 Storey""","""Steel""","""55""","""PVC""","""None""","""Brick""","""""","""Concrete""","""Engineered Hardwood""","""Copper, PEX, ABS""","""HWBB""","""Oil""","""Integrated with boiler""","""None""","""6""","""3""","""6""","""1530""","""""","""""","""1530 SqFt""","""47+/-yrs""","""2:0""","""Average"""
"""40 Bellroyal Crt Cole Harbour …","""Cole Harbour NS B2V 2B4""","""Apr/16/2025""","""Halifax Regional Municipality …","""See Schedule A and or Plot Map""","""6825 SqFt""","""SqFt""","""1985""","""Detached""","""Asphalt Shingle""","""12""","""4 Level Split""","""Wood""","""48""","""Vinyl""","""Part/FullFin'd+Crawl""","""Brick, Siding""","""728""","""Poured Concrete""","""Carpet""","""Copper, PEX, ABS""","""Electric Baseboard""","""Electric""","""Electric""","""None""","""7""","""3""","""7""","""1074""","""672""","""""","""1746 SqFt""","""40+/-""","""2:1""","""Average"""
"""10 Crofton Rd Toronto ON M4G2B…","""Toronto ON M4G2B4"

In [4]:
comps = []
for appriasal in data['appraisals']:
    for comp in appriasal['comps']:
        comps.append(comp)

comps_df = pl.from_dicts(comps)
print(comps_df.shape)
comps_df.head()

(264, 19)


distance_to_subject,prop_type,stories,address,city_province,sale_date,sale_price,dom,location_similarity,lot_size,age,condition,gla,room_count,bed_count,bath_count,basement_finish,parking,neighborhood
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""0.15 KM""","""Townhouse""","""2 Storey""","""930 Amberdale Cres""","""Kingston ON K7M 6V1""","""Oct/25/2024""","""378,900""","""38""","""Similar""","""N/A Condominium""","""49""","""Superior""","""1044 SqFt""","""6""","""3""","""2:0""","""Full/Finished""","""No Garage""",""""""
"""0.02 KM""","""Townhouse""","""2 Storey""","""771 Ashwood Dr""","""Kingston ON K7M 6X7""","""Feb/05/2025""","""327,000""","""51""","""Inferior""","""N/A Condominium""","""49""","""Inferior""","""1044 SqFt""","""6""","""3""","""1:0""","""Full/Finished""","""No Garage""",""""""
"""0.09 KM""","""Townhouse""","""2 Storey""","""995 Amberdale Cres""","""Kingston ON K7M 6X6""","""Feb/14/2025""","""315,000""","""95""","""Inferior""","""N/A Condominium""","""49""","""Inferior""","""1044 SqFt""","""6""","""3""","""2:0""","""Full/Part Finished""","""No Garage""",""""""
"""3.73 KM""","""Detached""","""1 Storey""","""64 Deermist Dr""","""Porters Lake NS B3E 1P3""","""Jan/16/2025""","""800,000""","""141+/-""","""Inferior""","""80212+/-SqFt""","""11+/-""","""Similar""","""1602+/-SqFt""","""6""","""3""","""2:0""","""Full/Finished""","""Dbl. Att. Gar.""",""""""
"""8.98 KM""","""Detached""","""1 Storey""","""85 Oceanic Dr""","""East Lawrencetown NS B2Z 1T6""","""Dec/21/2024""","""950,000""","""280+/-""","""Inferior""","""61700+/-SqFt""","""11+/-""","""Good""","""2100+/-SqFt""","""6""","""3""","""2:0""","""Full/Finished""","""1.5Att.Gar./Dbl.Att.Gar.""",""""""


In [5]:
properties = []
for appriasal in data['appraisals']:
    for property in appriasal['properties']:
        properties.append(property)

print(properties[0:5])
property_df = pl.from_dicts(properties,  infer_schema_length=1000)
print(property_df.shape)
property_df.head()

[{'id': 367, 'address': '463 Conservatory Dr', 'bedrooms': 3, 'gla': 1500, 'city': 'Kingston', 'province': 'Ontario', 'postal_code': 'K7M 9C8', 'property_sub_type': 'Detached', 'structure_type': 'Detached', 'style': 'Brick, Vinyl Siding', 'levels': 'Two', 'room_count': 11, 'full_baths': 3, 'half_baths': None, 'main_level_finished_area': None, 'upper_lvl_fin_area': None, 'bg_fin_area': None, 'lot_size_sf': 3555.5, 'year_built': None, 'roof': None, 'basement': 'Unfinished', 'cooling': 'Central Air', 'heating': 'Forced Air', 'close_price': 674000.0, 'close_date': '2025-01-13', 'public_remarks': "Welcome to this beautifully maintained and spacious family home! This charming residence features three generous bedrooms and two and a half bathrooms. Enjoy entertaining in the separate dining and living rooms, or relax in the open-concept kitchen that seamlessly flows into the inviting eat-in area and family room, complete with a cozy gas fireplace. Step outside through the sliding doors to disc

id,address,bedrooms,gla,city,province,postal_code,property_sub_type,structure_type,style,levels,room_count,full_baths,half_baths,main_level_finished_area,upper_lvl_fin_area,bg_fin_area,lot_size_sf,year_built,roof,basement,cooling,heating,close_price,close_date,public_remarks,latitude,longitude
i64,str,i64,i64,str,str,str,str,str,str,str,i64,i64,i64,f64,f64,null,f64,i64,str,str,str,str,f64,str,str,f64,f64
367,"""463 Conservatory Dr""",3,1500,"""Kingston""","""Ontario""","""K7M 9C8""","""Detached""","""Detached""","""Brick, Vinyl Siding""","""Two""",11,3,null,null,null,null,3555.5,null,null,"""Unfinished""","""Central Air""","""Forced Air""",674000.0,"""2025-01-13""","""Welcome to this beautifully ma…",44.2325,-76.5901
163443,"""463 Conservatory Drive """,3,1750,"""Kingston ""","""Ontario""","""K7M 9C8""","""Detached""","""Detached, 2-Storey ""","""2-Storey ""","""2-Storey """,11,null,null,null,null,null,3535.0,null,"""""","""Unfinished ""","""Central Air ""","""Forced Air """,674000.0,"""2025-01-13""","""Welcome to this beautifully ma…",44.2325,-76.5901
378,"""311 Janette St""",3,1500,"""Kingston""","""Ontario""","""K7P 0K8""","""Freehold Townhouse""","""Freehold Townhouse""","""2-Storey""","""Two""",11,4,null,null,null,null,2622.0,null,null,"""Fin W/O""","""Central Air""","""Forced Air""",585000.0,"""2025-01-14""","""Come and see what this fully f…",44.2622,-76.5904
130023,"""311 Janette Street """,3,1300,"""Kingston ""","""Ontario""","""K7P 0K8""","""Freehold Townhouse""","""Freehold Townhouse, 2-Storey ""","""2-Storey ""","""2-Storey """,11,null,null,null,null,null,2622.0,null,"""""","""Finished with Walk-Out ""","""Central Air ""","""Forced Air """,585000.0,"""2025-01-14""","""Come and see what this fully f…",44.2622,-76.5904
2782,"""4056 Bath Rd""",4,null,"""Kingston""","""Ontario""","""K7M 4Y4""","""Rural Resid""","""Rural Resid""","""2-Storey""",null,13,4,0,null,null,null,16672.0,null,null,"""Part Bsmt, Unfinished""",null,"""Baseboard""",685000.0,"""2025-01-15""","""Current valuation for 4056 Bat…",44.2407,-76.6102


In [6]:
def analyze_polars_dataframe(df: pl.DataFrame, display_limit_unique: int = 15, show_all_value_counts_limit: int = 50, top_n_value_counts: int = 5):
    """
    Performs and prints a basic analysis of each column in a Polars DataFrame.

    Args:
        df (pl.DataFrame): The Polars DataFrame to analyze.
        display_limit_unique (int): Max number of unique values to list directly.
                                    If more, a sample is shown.
        show_all_value_counts_limit (int): If unique values are below or equal to this for a
                                           categorical/string column, all value counts are shown.
        top_n_value_counts (int): If unique values exceed show_all_value_counts_limit for
                                  categorical/string columns, this many top value counts are shown.
    """
    if not isinstance(df, pl.DataFrame):
        print("Error: Input is not a Polars DataFrame.")
        return

    if df.is_empty():
        print("The DataFrame is empty. No analysis to perform.")
        return

    total_rows = df.height
    print(f"DataFrame Overview: {total_rows} rows, {df.width} columns\n")
    print("="*70)

    for col_name in df.columns:
        print(f"\n--- Analysis for Column: '{col_name}' ---")
        column_series = df[col_name]

        # Data Type
        print(f"Data Type: {column_series.dtype}")

        # Null Values
        null_count = column_series.null_count()
        non_null_count = total_rows - null_count
        null_percentage = (null_count / total_rows) * \
            100 if total_rows > 0 else 0
        print(f"Total Values (rows): {total_rows}")
        print(f"Non-Null Values: {non_null_count}")
        print(f"Null Values: {null_count} ({null_percentage:.2f}%)")

        # Unique Values
        num_unique_values = column_series.n_unique()  # Includes null if present
        print(
            f"Unique Values Count (includes null if present): {num_unique_values}")

        if num_unique_values > 0:
            # Get unique values, sort them. Sorting might fail for mixed types if not careful,
            # but generally okay for single-type columns or if Polars can handle it.
            try:
                unique_items = column_series.unique().sort()
                if num_unique_values <= display_limit_unique:
                    print(f"Unique Values List: {unique_items.to_list()}")
                else:
                    print(
                        f"Unique Values List (sample of first {display_limit_unique}): {unique_items.head(display_limit_unique).to_list()}")
                    # Check if None was in the head sample if nulls exist
                    if unique_items.null_count() > 0 and (None not in unique_items.head(display_limit_unique).drop_nulls().to_list() if unique_items.head(display_limit_unique).null_count() == 0 else None not in unique_items.head(display_limit_unique).to_list()):
                        print(
                            "Note: Null is also one of the unique values if present in the column and not shown in sample.")
            except Exception as e:
                print(
                    f"Could not display unique values list (possibly due to mixed types or other error): {e}")
        else:
            print(
                "Unique Values List: Column might be effectively empty or no distinct values.")
        print("\n" + "-"*50)
    print("\n" + "="*70)
    print("End of DataFrame Analysis.")

In [7]:
analyze_polars_dataframe(subjects_df)

DataFrame Overview: 88 rows, 35 columns


--- Analysis for Column: 'address' ---
Data Type: String
Total Values (rows): 88
Non-Null Values: 88
Null Values: 0 (0.00%)
Unique Values Count (includes null if present): 88
Unique Values List (sample of first 15): ['10 Crofton Rd Toronto ON M4G2B4', '10 Heritage Hills Dr Eastern Passage NS B3G1N5', '1004 Connery Ave Ottawa ON K1G2P1', '102 Stonewalk Dr Kemptville ON K0G 1J0', '1050 John F. Scott Rd Kingston ON K7L4V3', '1083 Cornerstone St NE Calgary AB T3N1G5', '11 Bashir St Maple ON L6A3A3', '11 Bermondsey Way Stittsville ON K2S2Y7', '11 PAUL AVE Ayr ON N0B1E0', '118 Colborne St Kingston ON K7K 1C9', '119 110 Auburn Meadows View SE Calgary, Alberta T3M 2M2', '12 Mcdougall St NW Langdon AB T0J1X2', '120 CASTLEGROVE RD NE Calgary AB T3J1T5', '1368 Shore Dr Bedford NS B4A2E8', '142-950 Oakview Ave Kingston ON K7M 6W8']

--------------------------------------------------

--- Analysis for Column: 'subject_city_province_zip' ---
Data Type: Strin

In [8]:
from typing import Dict
import polars as pl
import numpy as np
import random
import json

from clean_subject_impl import CleanSubjectImpl

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Load the data


def load_data(file_path: str) -> Dict:
    """Load JSON data from a file and return a dictionary."""
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


# Preprocess the data
cleanesd_subject_df: pl.DataFrame = CleanSubjectImpl().prep(load_data(
    'appraisals_dataset.json'))
print(cleanesd_subject_df.head(10))

INFO: Starting subject data preparation
INFO: Successfully extracted 88 subjects


shape: (10, 36)
┌─────────┬────────────┬────────────┬────────────┬───┬────────┬───────────┬────────────┬───────────┐
│ orderID ┆ address    ┆ subject_ci ┆ municipali ┆ … ┆ gla    ┆ num_baths ┆ effective_ ┆ property_ │
│ ---     ┆ ---        ┆ ty_provinc ┆ ty_distric ┆   ┆ ---    ┆ ---       ┆ date       ┆ class     │
│ str     ┆ str        ┆ e_zip      ┆ t          ┆   ┆ f64    ┆ f64       ┆ ---        ┆ ---       │
│         ┆            ┆ ---        ┆ ---        ┆   ┆        ┆           ┆ datetime[μ ┆ str       │
│         ┆            ┆ str        ┆ str        ┆   ┆        ┆           ┆ s]         ┆           │
╞═════════╪════════════╪════════════╪════════════╪═══╪════════╪═══════════╪════════════╪═══════════╡
│ 4762597 ┆ 142-950    ┆ "Twin Oak  ┆ Kingston   ┆ … ┆ 1044.0 ┆ 1.5       ┆ 2025-04-11 ┆ Basic     │
│         ┆ Oakview    ┆ Meadows"   ┆            ┆   ┆        ┆           ┆ 00:00:00   ┆           │
│         ┆ Ave        ┆            ┆            ┆   ┆        ┆           ┆

In [9]:
analyze_polars_dataframe(cleanesd_subject_df)

DataFrame Overview: 88 rows, 36 columns


--- Analysis for Column: 'orderID' ---
Data Type: String
Total Values (rows): 88
Non-Null Values: 88
Null Values: 0 (0.00%)
Unique Values Count (includes null if present): 88
Unique Values List (sample of first 15): ['4723364', '4742669', '4752865', '4755196', '4756192', '4756320', '4756829', '4757939', '4757966', '4758025', '4758316', '4758324', '4758529', '4758615', '4758672']

--------------------------------------------------

--- Analysis for Column: 'address' ---
Data Type: String
Total Values (rows): 88
Non-Null Values: 88
Null Values: 0 (0.00%)
Unique Values Count (includes null if present): 88
Unique Values List (sample of first 15): ['10 Crofton Rd Toronto ON M4G2B4', '10 Heritage Hills Dr Eastern Passage NS B3G1N5', '1004 Connery Ave Ottawa ON K1G2P1', '102 Stonewalk Dr Kemptville ON K0G 1J0', '1050 John F. Scott Rd Kingston ON K7L4V3', '1083 Cornerstone St NE Calgary AB T3N1G5', '11 Bashir St Maple ON L6A3A3', '11 Bermondsey Way Sti

In [10]:
analyze_polars_dataframe(comps_df)

DataFrame Overview: 264 rows, 19 columns


--- Analysis for Column: 'distance_to_subject' ---
Data Type: String
Total Values (rows): 264
Non-Null Values: 264
Null Values: 0 (0.00%)
Unique Values Count (includes null if present): 175
Unique Values List (sample of first 15): ['', '.05 km', '0.00 KM', '0.01 KM', '0.02 KM', '0.03 KM', '0.05 KM', '0.06 KM', '0.07 KM', '0.08 KM', '0.09 KM', '0.11 KM', '0.12 KM', '0.13 KM', '0.14 KM']

--------------------------------------------------

--- Analysis for Column: 'prop_type' ---
Data Type: String
Total Values (rows): 264
Non-Null Values: 261
Null Values: 3 (1.14%)
Unique Values Count (includes null if present): 10
Unique Values List: [None, 'Condominium', 'Detached', 'Duplex', 'Fourplex', 'High Rise Apartment', 'Low Rise Apartment', 'Semi Detached', 'Townhouse', 'Triplex']

--------------------------------------------------

--- Analysis for Column: 'stories' ---
Data Type: String
Total Values (rows): 264
Non-Null Values: 264
Null Values: 0 (0.

In [11]:
from clean_comp_impl import CleanCompImpl

cleaned_comps_df: pl.DataFrame = CleanCompImpl().prep(data)
cleaned_comps_df.head()

INFO: Starting comparable property data preparation
INFO: Successfully extracted 264 comparable properties


orderID,address,city_province,prop_type,condition,basement_finish,parking,neighborhood,stories,distance_to_subject,sale_price,dom,lot_size,age,gla,room_count,bed_count,bath_count,location_similarity,sale_date,price_per_sqft,property_class,sale_year,sale_month,sale_quarter,lot_size_unit
str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,datetime[μs],f64,str,i32,i8,i8,str
"""4762597""","""930 Amberdale Cres""","""Kingston ON K7M 6V1""","""Townhouse""","""Superior""","""Full/Finished""","""No Garage""","""""","""2 Storey""",0.15,378900.0,38.0,2680.0,49.0,1044.0,6.0,3.0,2.0,"""Similar""",2024-10-25 00:00:00,362.931034,"""Basic""",2024,10,4,"""SqM"""
"""4762597""","""771 Ashwood Dr""","""Kingston ON K7M 6X7""","""Townhouse""","""Inferior""","""Full/Finished""","""No Garage""","""""","""2 Storey""",0.02,327000.0,51.0,2680.0,49.0,1044.0,6.0,3.0,1.0,"""Inferior""",2025-02-05 00:00:00,313.218391,"""Basic""",2025,2,1,"""SqM"""
"""4762597""","""995 Amberdale Cres""","""Kingston ON K7M 6X6""","""Townhouse""","""Inferior""","""Full/Part Finished""","""No Garage""","""""","""2 Storey""",0.09,315000.0,95.0,2680.0,49.0,1044.0,6.0,3.0,2.0,"""Inferior""",2025-02-14 00:00:00,301.724138,"""Basic""",2025,2,1,"""SqM"""
"""4762739""","""64 Deermist Dr""","""Porters Lake NS B3E 1P3""","""Detached""","""Similar""","""Full/Finished""","""Dbl. Att. Gar.""","""""","""1 Storey""",3.73,800000.0,141.0,2680.0,11.0,1560.55,6.0,3.0,2.0,"""Inferior""",2025-01-16 00:00:00,429.50237,"""Basic""",2025,1,1,"""SqM"""
"""4762739""","""85 Oceanic Dr""","""East Lawrencetown NS B2Z 1T6""","""Detached""","""Good""","""Full/Finished""","""1.5Att.Gar./Dbl.Att.Gar.""","""""","""1 Storey""",8.98,950000.0,280.0,2680.0,11.0,1560.55,6.0,3.0,2.0,"""Inferior""",2024-12-21 00:00:00,429.50237,"""Standard""",2024,12,4,"""SqM"""


In [12]:
analyze_polars_dataframe(cleaned_comps_df)

DataFrame Overview: 264 rows, 26 columns


--- Analysis for Column: 'orderID' ---
Data Type: String
Total Values (rows): 264
Non-Null Values: 264
Null Values: 0 (0.00%)
Unique Values Count (includes null if present): 88
Unique Values List (sample of first 15): ['4723364', '4742669', '4752865', '4755196', '4756192', '4756320', '4756829', '4757939', '4757966', '4758025', '4758316', '4758324', '4758529', '4758615', '4758672']

--------------------------------------------------

--- Analysis for Column: 'address' ---
Data Type: String
Total Values (rows): 264
Non-Null Values: 264
Null Values: 0 (0.00%)
Unique Values Count (includes null if present): 258
Unique Values List (sample of first 15): ['1 Partridge Nest Dr', '1 Rock Lake Pt NW', '1 Virginia Dr', '1-150 Broadway', '1-21 Cherry St', '100 Brambling Way', '102 Deeridge Rd', '1035 Connery Ave', '105-22 Auburn Bay Link SE', '1059 Cornerstone St NE', '10673 Hidden Valley Dr NW', '107 Ahmadi Cres', '108 Macewan Park Green NW', '109 Kinnib

In [13]:
analyze_polars_dataframe(property_df)

DataFrame Overview: 9820 rows, 28 columns


--- Analysis for Column: 'id' ---
Data Type: Int64
Total Values (rows): 9820
Non-Null Values: 9820
Null Values: 0 (0.00%)
Unique Values Count (includes null if present): 7555
Unique Values List (sample of first 15): [2, 8, 26, 28, 30, 36, 37, 42, 43, 46, 50, 51, 52, 53, 54]

--------------------------------------------------

--- Analysis for Column: 'address' ---
Data Type: String
Total Values (rows): 9820
Non-Null Values: 9820
Null Values: 0 (0.00%)
Unique Values Count (includes null if present): 7536
Unique Values List (sample of first 15): ['(Address not available) ', '1 Dove Place ', '1 Exeter Avenue ', '1 Grandlea Terrace ', '1 Grandview Court', '1 Hollowood Court ', '1 Lancaster Drive', '1 Le May Road ', '1 Milloy Place ', '1 Partridge Nest Drive', '1 Place Madeleine Street ', '1 Rock Lake Point NW ', '1 Royal Elm Way NW ', '1 Saturn Drive', '1 Shaw Crescent']

--------------------------------------------------

--- Analysis for Column

In [14]:
from clean_property_impl import CleanPropertyImpl


cleaned_property_df: pl.DataFrame = CleanPropertyImpl().prep(data)
cleaned_comps_df.head(10)

INFO: Starting property data preparation
INFO: Successfully extracted 9820 properties


orderID,address,city_province,prop_type,condition,basement_finish,parking,neighborhood,stories,distance_to_subject,sale_price,dom,lot_size,age,gla,room_count,bed_count,bath_count,location_similarity,sale_date,price_per_sqft,property_class,sale_year,sale_month,sale_quarter,lot_size_unit
str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,datetime[μs],f64,str,i32,i8,i8,str
"""4762597""","""930 Amberdale Cres""","""Kingston ON K7M 6V1""","""Townhouse""","""Superior""","""Full/Finished""","""No Garage""","""""","""2 Storey""",0.15,378900.0,38.0,2680.0,49.0,1044.0,6.0,3.0,2.0,"""Similar""",2024-10-25 00:00:00,362.931034,"""Basic""",2024,10,4,"""SqM"""
"""4762597""","""771 Ashwood Dr""","""Kingston ON K7M 6X7""","""Townhouse""","""Inferior""","""Full/Finished""","""No Garage""","""""","""2 Storey""",0.02,327000.0,51.0,2680.0,49.0,1044.0,6.0,3.0,1.0,"""Inferior""",2025-02-05 00:00:00,313.218391,"""Basic""",2025,2,1,"""SqM"""
"""4762597""","""995 Amberdale Cres""","""Kingston ON K7M 6X6""","""Townhouse""","""Inferior""","""Full/Part Finished""","""No Garage""","""""","""2 Storey""",0.09,315000.0,95.0,2680.0,49.0,1044.0,6.0,3.0,2.0,"""Inferior""",2025-02-14 00:00:00,301.724138,"""Basic""",2025,2,1,"""SqM"""
"""4762739""","""64 Deermist Dr""","""Porters Lake NS B3E 1P3""","""Detached""","""Similar""","""Full/Finished""","""Dbl. Att. Gar.""","""""","""1 Storey""",3.73,800000.0,141.0,2680.0,11.0,1560.55,6.0,3.0,2.0,"""Inferior""",2025-01-16 00:00:00,429.50237,"""Basic""",2025,1,1,"""SqM"""
"""4762739""","""85 Oceanic Dr""","""East Lawrencetown NS B2Z 1T6""","""Detached""","""Good""","""Full/Finished""","""1.5Att.Gar./Dbl.Att.Gar.""","""""","""1 Storey""",8.98,950000.0,280.0,2680.0,11.0,1560.55,6.0,3.0,2.0,"""Inferior""",2024-12-21 00:00:00,429.50237,"""Standard""",2024,12,4,"""SqM"""
"""4762739""","""270 Quinlan Dr""","""Head of Jeddore NS B0J 1P0""","""Detached""","""Good""","""Full/Part Finished""","""Driveway""","""""","""1.5 Storey""",17.33,829900.0,47.0,2680.0,12.0,1560.55,5.0,2.0,2.0,"""Similar""",2024-11-23 00:00:00,429.50237,"""Standard""",2024,11,4,"""SqM"""
"""4763065""","""6 Marshall Ave""","""Ayr ON N0B 1E0""","""Detached""","""Superior""","""Rec Room""","""Dbl. Att. Gar.""","""""","""2 Storey""",0.31,1.15e6,7.0,15116.0,58.0,2003.0,8.0,4.0,2.5,"""Similar""",2025-03-26 00:00:00,574.138792,"""Basic""",2025,3,1,"""SqFt"""
"""4763065""","""3219 Roseville Rd""","""Ayr ON N0B 1E0""","""Detached""","""Superior""","""Rec Room""","""Dbl. Att. Gar.""","""""","""2 Storey""",0.5,1.13e6,40.0,11138.0,1.0,2940.0,8.0,4.0,2.5,"""Inferior""",2024-12-24 00:00:00,384.353741,"""Basic""",2024,12,4,"""SqFt"""
"""4763065""","""13 James Crt""","""Heidelberg ON N0B 2M1""","""Detached""","""Average""","""Rec/Bath""","""Dbl. Att. Gar.""","""""","""2 Storey""",23.23,1.22e6,6.0,29200.0,40.0,2365.0,7.0,3.0,2.5,"""Superior""",2025-04-16 00:00:00,515.856237,"""Standard""",2025,4,2,"""SqFt"""


In [15]:
analyze_polars_dataframe(cleaned_property_df)

DataFrame Overview: 9820 rows, 35 columns


--- Analysis for Column: 'id' ---
Data Type: Int64
Total Values (rows): 9820
Non-Null Values: 9820
Null Values: 0 (0.00%)
Unique Values Count (includes null if present): 7555
Unique Values List (sample of first 15): [2, 8, 26, 28, 30, 36, 37, 42, 43, 46, 50, 51, 52, 53, 54]

--------------------------------------------------

--- Analysis for Column: 'orderID' ---
Data Type: String
Total Values (rows): 9820
Non-Null Values: 9820
Null Values: 0 (0.00%)
Unique Values Count (includes null if present): 88
Unique Values List (sample of first 15): ['4723364', '4742669', '4752865', '4755196', '4756192', '4756320', '4756829', '4757939', '4757966', '4758025', '4758316', '4758324', '4758529', '4758615', '4758672']

--------------------------------------------------

--- Analysis for Column: 'address' ---
Data Type: String
Total Values (rows): 9820
Non-Null Values: 9820
Null Values: 0 (0.00%)
Unique Values Count (includes null if present): 7536
Unique V

In [16]:
print(f"cleanesd_subject_df.columns: {cleanesd_subject_df.columns}")
print("************************************************")
print(f"cleaned_comps_df.columns: {cleaned_comps_df.columns}")
print("************************************************")
print(f"cleaned_property_df.columns: {cleaned_property_df.columns}")

cleanesd_subject_df.columns: ['orderID', 'address', 'subject_city_province_zip', 'municipality_district', 'structure_type', 'style', 'construction', 'basement', 'exterior_finish', 'foundation_walls', 'flooring', 'plumbing_lines', 'heating', 'fuel_type', 'water_heater', 'cooling', 'condition', 'roofing', 'windows', 'unit_measurement', 'site_dimensions', 'lot_size_sf', 'year_built', 'effective_age', 'remaining_economic_life', 'basement_area', 'room_count', 'num_beds', 'room_total', 'main_lvl_area', 'second_lvl_area', 'third_lvl_area', 'gla', 'num_baths', 'effective_date', 'property_class']
************************************************
cleaned_comps_df.columns: ['orderID', 'address', 'city_province', 'prop_type', 'condition', 'basement_finish', 'parking', 'neighborhood', 'stories', 'distance_to_subject', 'sale_price', 'dom', 'lot_size', 'age', 'gla', 'room_count', 'bed_count', 'bath_count', 'location_similarity', 'sale_date', 'price_per_sqft', 'property_class', 'sale_year', 'sale_month

In [17]:
subjects_df = pl.read_csv('subject_df.csv')
comps_df = pl.read_csv('comps_df.csv')
property_df = pl.read_csv('property_df.csv')

print(subjects_df.shape)
print(comps_df.shape)
print(property_df.shape)


(88, 40)
(264, 30)
(9820, 37)


In [18]:
print(f"cleanesd_subject_df.columns: {subjects_df.columns}")
print("************************************************")
print(f"cleaned_comps_df.columns: {comps_df.columns}")
print("************************************************")
print(f"cleaned_property_df.columns: {property_df.columns}")

cleanesd_subject_df.columns: ['orderID', 'address', 'subject_city_province_zip', 'municipality_district', 'structure_type', 'style', 'construction', 'basement', 'exterior_finish', 'foundation_walls', 'flooring', 'plumbing_lines', 'heating', 'fuel_type', 'water_heater', 'cooling', 'condition', 'roofing', 'windows', 'unit_measurement', 'site_dimensions', 'lot_size_sf', 'year_built', 'effective_age', 'remaining_economic_life', 'basement_area', 'room_count', 'num_beds', 'room_total', 'main_lvl_area', 'second_lvl_area', 'third_lvl_area', 'gla', 'num_baths', 'effective_date', 'property_class', 'full_address', 'normalized_address', 'calculated_latitude', 'calculated_longitude']
************************************************
cleaned_comps_df.columns: ['orderID', 'address', 'city_province', 'prop_type', 'condition', 'basement_finish', 'parking', 'neighborhood', 'stories', 'distance_to_subject', 'sale_price', 'dom', 'lot_size', 'age', 'gla', 'room_count', 'bed_count', 'bath_count', 'location_s

1. remove all subjects with invalid addresses.
2. filter the comps that are associated with the order_id's of filtered subjects.
3. Compute ranking dataset.

In [19]:
filtered_subjects_df = subjects_df.filter(pl.col("calculated_latitude").is_not_null())
filtered_subjects_df.head(2)

orderID,address,subject_city_province_zip,municipality_district,structure_type,style,construction,basement,exterior_finish,foundation_walls,flooring,plumbing_lines,heating,fuel_type,water_heater,cooling,condition,roofing,windows,unit_measurement,site_dimensions,lot_size_sf,year_built,effective_age,remaining_economic_life,basement_area,room_count,num_beds,room_total,main_lvl_area,second_lvl_area,third_lvl_area,gla,num_baths,effective_date,property_class,full_address,normalized_address,calculated_latitude,calculated_longitude
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,f64,f64
4762597,"""142-950 Oakview Ave Kingston O…","""""Twin Oak Meadows""""","""Kingston""","""Townhouse""","""2 Storey""","""Wood Frame""","""Full/Finished""","""Brick Veneer, Vinyl Siding""","""Concrete""","""Hardwood, carpet, laminate, ce…","""Copper, PEX, ABS""","""Forced Air""","""Natural Gas""","""40 Gallon - Gas""","""Central Air""","""Average""","""Asphalt Shingle""","""Vinyl""","""SqFt""","""Condo Common Property""",3251.5,1976.0,25.0,50.0,522.0,6.0,3.0,6.0,522.0,522.0,300.0,1044.0,1.5,"""2025-04-11T00:00:00.000000""","""Basic""","""142-950 Oakview Ave Kingston O…","""Oakview Avenue, Bayridge, King…",44.248478,-76.589595
4762739,"""7180 207 HWY Halifax NS B0J2L0""","""West Chezzetcook, NS B0J2L0""","""Halifax Regional Municipality …","""Detached""","""1.5 Storey""","""Log""","""Full/Finished""","""Log""","""Poured Concrete""","""Carpet""","""Copper, PEX, ABS""","""Radiant""","""Electric""","""80 +/- gl Electric""","""Ductless mini split""","""Average""","""Asphalt Shingle""","""Wood""","""SqFt""","""See Schedule A and or Plot Map""",3251.5,2011.0,10.0,50.0,1060.0,6.0,3.0,6.0,1060.0,440.0,300.0,1500.0,2.5,"""2025-04-17T00:00:00.000000""","""Basic""","""7180 207 HWY Halifax NS B0J2L0""","""7180, Highway 207, Kada Estate…",44.716994,-63.258544


In [20]:
filtered_comps_df = comps_df.filter(pl.col("orderID").is_in(filtered_subjects_df["orderID"].unique()))
print(filtered_comps_df.shape)
filtered_comps_df.head(2)

(207, 30)


orderID,address,city_province,prop_type,condition,basement_finish,parking,neighborhood,stories,distance_to_subject,sale_price,dom,lot_size,age,gla,room_count,bed_count,bath_count,location_similarity,sale_date,price_per_sqft,property_class,sale_year,sale_month,sale_quarter,lot_size_unit,full_address,normalized_address,calculated_latitude,calculated_longitude
i64,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,str,i64,i64,i64,str,str,str,f64,f64
4762597,"""930 Amberdale Cres""","""Kingston ON K7M 6V1""","""Townhouse""","""Superior""","""Full/Finished""","""No Garage""","""""","""2 Storey""",0.15,378900.0,38.0,2680.0,49.0,1044.0,6.0,3.0,2.0,"""Similar""","""2024-10-25T00:00:00.000000""",362.931034,"""Basic""",2024,10,4,"""SqM""","""930 Amberdale Cres, Kingston O…","""930, Amberdale Crescent, Bayri…",44.249366,-76.587649
4762597,"""771 Ashwood Dr""","""Kingston ON K7M 6X7""","""Townhouse""","""Inferior""","""Full/Finished""","""No Garage""","""""","""2 Storey""",0.02,327000.0,51.0,2680.0,49.0,1044.0,6.0,3.0,1.0,"""Inferior""","""2025-02-05T00:00:00.000000""",313.218391,"""Basic""",2025,2,1,"""SqM""","""771 Ashwood Dr, Kingston ON K7…","""771, Ashwood Drive, Bayridge, …",44.249302,-76.588428


In [21]:
# Remove duplicates from comps_df
filtered_comps_df = filtered_comps_df.unique(subset=["orderID", "normalized_address"], keep="first", maintain_order=True)
print(filtered_comps_df.shape)


(204, 30)


In [22]:
filtered_property_df = property_df.filter(pl.col("orderID").is_in(filtered_subjects_df["orderID"].unique()))
print(filtered_property_df.shape)
filtered_property_df.head(2)

(6883, 37)


id,orderID,address,city,province,postal_code,latitude,longitude,property_sub_type,structure_type,style,levels,bedrooms,full_baths,half_baths,room_count,gla,main_level_finished_area,upper_lvl_fin_area,lot_size_sf,year_built,roof,basement,cooling,heating,close_price,close_date,public_remarks,total_baths,price_per_sqft,property_class,sale_year,sale_month,sale_quarter,lot_size_unit,full_address,normalized_address
i64,i64,str,str,str,str,f64,f64,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,f64,str,str,f64,f64,str,i64,i64,i64,str,str,str
367,4762597,"""463 Conservatory Dr""","""Kingston""","""Ontario""","""K7M 9C8""",44.2325,-76.5901,"""Detached""","""Detached""","""Brick, Vinyl Siding""","""Two""",3.0,3.0,1.0,11.0,1500.0,843.0,850.0,3555.5,2004.0,"""""","""Unfinished""","""Central Air""","""Forced Air""",674000.0,"""2025-01-13T00:00:00.000000""","""Welcome to this beautifully ma…",2.5,449.333333,"""Standard""",2025,1,1,"""SqM""","""463 Conservatory Dr, Kingston,…","""443, Conservatory Drive, Conse…"
163443,4762597,"""463 Conservatory Drive ""","""Kingston ""","""Ontario""","""K7M 9C8""",44.2325,-76.5901,"""Detached""","""Detached, 2-Storey ""","""2-Storey ""","""2-Storey """,3.0,2.0,1.0,11.0,1750.0,843.0,850.0,3535.0,2004.0,"""""","""Unfinished ""","""Central Air ""","""Forced Air """,674000.0,"""2025-01-13T00:00:00.000000""","""Welcome to this beautifully ma…",2.5,385.142857,"""Standard""",2025,1,1,"""SqM""","""463 Conservatory Drive , Kings…","""443, Conservatory Drive, Conse…"


In [23]:
filtered_property_df = filtered_property_df.unique(subset=["orderID", "normalized_address", "latitude", "longitude"], keep="first", maintain_order=True)
print(filtered_property_df.shape)


(5602, 37)


In [24]:
filtered_comps_df = filtered_comps_df.join(
    filtered_property_df.select(["orderID", "normalized_address", "cooling","heating"]),
    on=["orderID", "normalized_address"],
    how="left"
).with_columns(
    pl.col("cooling").fill_null("not found").alias("cooling"),
    pl.col("heating").fill_null("not found").alias("heating"),
)


In [25]:
filtered_comps_df.head(2)

orderID,address,city_province,prop_type,condition,basement_finish,parking,neighborhood,stories,distance_to_subject,sale_price,dom,lot_size,age,gla,room_count,bed_count,bath_count,location_similarity,sale_date,price_per_sqft,property_class,sale_year,sale_month,sale_quarter,lot_size_unit,full_address,normalized_address,calculated_latitude,calculated_longitude,cooling,heating
i64,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,str,i64,i64,i64,str,str,str,f64,f64,str,str
4762597,"""930 Amberdale Cres""","""Kingston ON K7M 6V1""","""Townhouse""","""Superior""","""Full/Finished""","""No Garage""","""""","""2 Storey""",0.15,378900.0,38.0,2680.0,49.0,1044.0,6.0,3.0,2.0,"""Similar""","""2024-10-25T00:00:00.000000""",362.931034,"""Basic""",2024,10,4,"""SqM""","""930 Amberdale Cres, Kingston O…","""930, Amberdale Crescent, Bayri…",44.249366,-76.587649,"""not found""","""not found"""
4762597,"""771 Ashwood Dr""","""Kingston ON K7M 6X7""","""Townhouse""","""Inferior""","""Full/Finished""","""No Garage""","""""","""2 Storey""",0.02,327000.0,51.0,2680.0,49.0,1044.0,6.0,3.0,1.0,"""Inferior""","""2025-02-05T00:00:00.000000""",313.218391,"""Basic""",2025,2,1,"""SqM""","""771 Ashwood Dr, Kingston ON K7…","""771, Ashwood Drive, Bayridge, …",44.249302,-76.588428,"""not found""","""not found"""


In [26]:
filtered_subjects_df = filtered_subjects_df.with_columns(
    age=2025 - pl.col("year_built")
)
filtered_property_df = filtered_property_df.with_columns(
    age=2025 - pl.col("year_built")
)


In [27]:
filtered_comps_df["lot_size_unit"].unique()

lot_size_unit
str
"""SqFt"""
"""SqM"""


In [28]:
filtered_subjects_df["unit_measurement"].unique()

unit_measurement
str
"""N/A"""
"""Acres"""
"""SqM"""
null
"""SqFt"""


In [29]:
filtered_property_df["lot_size_unit"].unique()

lot_size_unit
str
"""Acre"""
"""SqM"""
"""SqFt"""


In [30]:
filtered_subjects_df = filtered_subjects_df.with_columns(
    pl.col("unit_measurement").fill_null("N/A"))
filtered_subjects_df = filtered_subjects_df.with_columns({
    "unit_measurement": pl.when(pl.col("unit_measurement") == "SqM").then("SqM")
    .when(pl.col("unit_measurement") == "SqFt").then("SqFt")
    .when(pl.col("unit_measurement") == "Acres").then("Acre")
    .otherwise(pl.col("unit_measurement"))
})

In [31]:
filtered_comps_df = filtered_comps_df.with_columns({
    "lot_size_unit": pl.when(pl.col("lot_size_unit") == "SqM").then("SqM")
    .when(pl.col("lot_size_unit") == "SqFt").then("SqFt")
    .when(pl.col("lot_size_unit") == "Acre").then("Acre")
    .otherwise(pl.col("lot_size_unit"))
})

In [32]:
filterted_property_df = filtered_property_df.with_columns({
    "lot_size_unit": pl.when(pl.col("lot_size_unit") == "SqM").then("SqM")
    .when(pl.col("lot_size_unit") == "SqFt").then("SqFt")
    .when(pl.col("lot_size_unit") == "Acre").then("Acre")
    .otherwise(pl.col("lot_size_unit"))
})

In [33]:
def standardize_lot_size_to_sqft(df, lot_size_column, unit_column):
    """
    Standardizes lot size values to square feet based on the unit measurement.
    
    Parameters:
    -----------
    df : pl.DataFrame
        The dataframe containing the lot size and unit columns
    lot_size_column : str
        The name of the column containing lot size values
    unit_column : str
        The name of the column containing unit measurements (SqFt, SqM, Acre, etc.)
    
    Returns:
    --------
    pl.DataFrame
        The modified dataframe with standardized lot size values in square feet
    """
    # Make a copy to avoid modifying the original
    result_df = df.clone()

    # Ensure the unit column has no nulls
    result_df = result_df.with_columns(pl.col(unit_column).fill_null("N/A"))

    # Create a new column with the converted values
    result_df = result_df.with_columns(
        pl.when(pl.col(unit_column) == "SqM")
        .then(pl.col(lot_size_column) * 10.764)  # 1 sq meter = 10.764 sq feet
        .when(pl.col(unit_column) == "Acre")
        .then(pl.col(lot_size_column) * 43560)   # 1 acre = 43,560 sq feet
        # Keep original for SqFt or N/A
        .otherwise(pl.col(lot_size_column))
        .alias(lot_size_column)                  # Replace the original column
    )

    return result_df

In [34]:
filtered_subjects_df = standardize_lot_size_to_sqft(
    filtered_subjects_df,
    lot_size_column="lot_size_sf",
    unit_column="unit_measurement"
)

filtered_comps_df = standardize_lot_size_to_sqft(
    filtered_comps_df,
    lot_size_column="lot_size",
    unit_column="lot_size_unit"
)

filtered_property_df = standardize_lot_size_to_sqft(
    filtered_property_df,
    lot_size_column="lot_size_sf",
    unit_column="lot_size_unit"
)


In [35]:
from geopy.distance import geodesic

In [36]:
# Prepare subject table.
subject = (
    filtered_subjects_df.select(
        [
            "orderID",
            "style",
            "heating",
            "cooling",
            "unit_measurement",
            "lot_size_sf",
            "age",
            "room_count",
            "num_beds",
            "gla",
            "num_baths",
            "property_class",
            "calculated_latitude",
            "calculated_longitude",
        ]
    )
    .rename({
        "style": "style_sub",
        "heating": "heating_sub",
        "cooling": "cooling_sub",
        "unit_measurement": "unit_measurement_sub",
        "lot_size_sf": "lot_size_sf_sub",
        "age": "age_sub",
        "room_count": "room_count_sub",
        "num_beds": "num_beds_sub",
        "gla": "gla_sub",
        "num_baths": "num_baths_sub",
        "property_class": "property_class_sub",
        "calculated_latitude": "latitude_sub",
        "calculated_longitude": "longitude_sub",
    })
)

# Build positives.
positives = (
    filtered_comps_df.select(
        [
            "orderID",
            "stories",
            "heating",
            "cooling",
            "lot_size_unit",
            "lot_size",
            "age",
            "room_count",
            "bed_count",
            "gla",
            "bath_count",
            "property_class",
            "calculated_latitude",
            "calculated_longitude",
        ]
    )
    .rename({
        "stories": "style_cand",
        "heating": "heating_cand",
        "cooling": "cooling_cand",
        "lot_size_unit": "unit_measurement_cand",
        "lot_size": "lot_size_sf_cand",
        "age": "age_cand",
        "room_count": "room_count_cand",
        "bed_count": "num_beds_cand",
        "gla": "gla_cand",
        "bath_count": "num_baths_cand",
        "property_class": "property_class_cand",
        "calculated_latitude": "latitude_cand",
        "calculated_longitude": "longitude_cand"
    }).join(subject, on="orderID").with_columns(
        pl.lit(1).alias("label")
    )
)

# Build negatives.
neg_list = []
N_NEG = 10
for order_id in subject["orderID"].unique().to_list():
    subject_row = subject.filter(pl.col("orderID") == order_id)
    pool = (
        filtered_property_df.filter(pl.col("orderID") == order_id)
        .select([
            "orderID",
            "normalized_address",  # Keep this for filtering
            "style",
            "heating",
            "cooling",
            "lot_size_unit",
            "lot_size_sf",
            "age",
            "room_count",
            "bedrooms",
            "gla",
            "total_baths",
            "property_class",
            "latitude",
            "longitude"
        ])
        .rename({
            "style": "style_cand",
            "heating": "heating_cand",
            "cooling": "cooling_cand",
            "lot_size_unit": "unit_measurement_cand",
            "lot_size_sf": "lot_size_sf_cand",
            "age": "age_cand",
            "room_count": "room_count_cand",
            "bedrooms": "num_beds_cand",
            "gla": "gla_cand",
            "total_baths": "num_baths_cand",
            "property_class": "property_class_cand",
            "latitude": "latitude_cand",
            "longitude": "longitude_cand",
        })
    )
    choosen = filtered_comps_df.filter(pl.col("orderID") == order_id).get_column("normalized_address").unique()
    pool = pool.filter(~pl.col("normalized_address").is_in(choosen))
    sample_size = min(N_NEG, pool.height)
    sampled = pool.sample(sample_size, seed=42)
    neg_list.append(sampled.join(subject_row, on="orderID").with_columns(pl.lit(0).alias("label")))


negatives = pl.concat(neg_list).drop(["normalized_address"])
print(positives.columns)
print(negatives.columns)

['orderID', 'style_cand', 'heating_cand', 'cooling_cand', 'unit_measurement_cand', 'lot_size_sf_cand', 'age_cand', 'room_count_cand', 'num_beds_cand', 'gla_cand', 'num_baths_cand', 'property_class_cand', 'latitude_cand', 'longitude_cand', 'style_sub', 'heating_sub', 'cooling_sub', 'unit_measurement_sub', 'lot_size_sf_sub', 'age_sub', 'room_count_sub', 'num_beds_sub', 'gla_sub', 'num_baths_sub', 'property_class_sub', 'latitude_sub', 'longitude_sub', 'label']
['orderID', 'style_cand', 'heating_cand', 'cooling_cand', 'unit_measurement_cand', 'lot_size_sf_cand', 'age_cand', 'room_count_cand', 'num_beds_cand', 'gla_cand', 'num_baths_cand', 'property_class_cand', 'latitude_cand', 'longitude_cand', 'style_sub', 'heating_sub', 'cooling_sub', 'unit_measurement_sub', 'lot_size_sf_sub', 'age_sub', 'room_count_sub', 'num_beds_sub', 'gla_sub', 'num_baths_sub', 'property_class_sub', 'latitude_sub', 'longitude_sub', 'label']


In [37]:
print(positives.shape)
print(negatives.shape)

df = pl.concat([positives, negatives])
print(df.shape)


(209, 28)
(686, 28)
(895, 28)


In [38]:
print(df.columns)

['orderID', 'style_cand', 'heating_cand', 'cooling_cand', 'unit_measurement_cand', 'lot_size_sf_cand', 'age_cand', 'room_count_cand', 'num_beds_cand', 'gla_cand', 'num_baths_cand', 'property_class_cand', 'latitude_cand', 'longitude_cand', 'style_sub', 'heating_sub', 'cooling_sub', 'unit_measurement_sub', 'lot_size_sf_sub', 'age_sub', 'room_count_sub', 'num_beds_sub', 'gla_sub', 'num_baths_sub', 'property_class_sub', 'latitude_sub', 'longitude_sub', 'label']


In [39]:
numerical_ranking = df.with_columns(
    pl.struct(["latitude_sub", "longitude_sub", "latitude_cand", "longitude_cand"]).
    map_elements(lambda r: geodesic((r["latitude_sub"], r["longitude_sub"]),
                                    (r["latitude_cand"], r["longitude_cand"])).km).alias("dist_km"),
    (pl.col("room_count_sub").abs() - pl.col("room_count_cand").abs()).alias("room_diff"),
    (pl.col("num_beds_sub").abs() - pl.col("num_beds_cand").abs()
     ).alias("bed_diff"),  # Changed from bed_count_sub
    (pl.col("num_baths_sub") - pl.col("num_baths_cand").abs()
     ).alias("bath_diff"),  # Changed from bath_count_sub
    (pl.col("lot_size_sf_sub").abs() - pl.col("lot_size_sf_cand").abs()).alias("lot_diff"),
    (pl.col("age_sub").abs() - pl.col("age_cand").abs()).alias("age_diff"),
    (pl.col("gla_sub").abs() - pl.col("gla_cand").abs()).alias("gla_diff"),
)

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

In [40]:
numerical_ranking["property_class_sub"].unique()

property_class_sub
str
"""Premium"""
"""Standard"""
"""Basic"""
"""Luxury"""


In [41]:
from rapidfuzz import fuzz
import polars as pl

# Define the features separately
categorical_features = [
    ("style_sub", "style_cand", "style_sim"),
    ("heating_sub", "heating_cand", "heating_sim"),
    ("cooling_sub", "cooling_cand", "cooling_sim"),
    ("property_class_sub", "property_class_cand", "property_class_sim")
]

# Add categorical features one by one
final_df = numerical_ranking.clone()

for col1, col2, output_name in categorical_features:
    try:
        # Create a new column using a different approach
        final_df = final_df.with_columns([
            pl.struct([pl.col(col1), pl.col(col2)])
            .map_elements(
                lambda row: fuzz.ratio(
                    str(row[col1]) if row[col1] is not None else "",
                    str(row[col2]) if row[col2] is not None else ""
                ) / 100.0,
                return_dtype=pl.Float64
            )
            .alias(output_name)
        ])
        print(f"Successfully added {output_name}")
    except Exception as e:
        print(f"Error adding {output_name}: {e}")

Successfully added style_sim
Successfully added heating_sim
Successfully added cooling_sim
Successfully added property_class_sim


In [42]:
final_df.shape

(895, 39)

In [43]:
features = ['dist_km',
 'room_diff',
 'bed_diff',
 'bath_diff',
 'lot_diff',
 'age_diff',
 'gla_diff',
 'style_sim',
 'heating_sim',
 'cooling_sim',
 'property_class_sim']

train_pdf = final_df.select(["orderID", *features, "label"]).to_pandas()
groups = train_pdf.groupby("orderID").size().to_numpy()
X, y = train_pdf[features], train_pdf["label"]

In [44]:
from xgboost import XGBClassifier, XGBRanker

In [45]:
pt_model = XGBClassifier(
    objective="binary:logistic",
    use_label_encoder=False,
    eval_metric="logloss",
    learning_rate=0.05,
    n_estimators=200,
    random_state=42
)
pt_model.fit(X, y)

/Users/jyothivishnuvardhankolla/anaconda3/envs/rec-sys/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [21:07:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [46]:
train_pdf["pt_score"] = pt_model.predict_proba(X)[:, 1]

In [47]:
train_pdf[train_pdf["label"] == 1].head(10)

,orderID,dist_km,room_diff,bed_diff,bath_diff,lot_diff,age_diff,gla_diff,style_sim,heating_sim,cooling_sim,property_class_sim,label,pt_score
0,4762597,0.184041,0.0,0.0,-0.5,-25596.02,0.0,0.00,1.000000,0.210526,0.300000,1.000000,1,0.998717
1,4762597,0.130678,0.0,0.0,0.5,-25596.02,0.0,0.00,1.000000,0.210526,0.300000,1.000000,1,0.998717
2,4762597,0.094737,0.0,0.0,-0.5,-25596.02,0.0,0.00,1.000000,0.210526,0.300000,1.000000,1,0.998717
3,4762739,3.786834,0.0,0.0,0.5,-25596.02,3.0,-60.55,0.888889,0.250000,0.214286,1.000000,1,0.968184
4,4762739,9.068734,0.0,0.0,0.5,-25596.02,3.0,-60.55,0.888889,0.250000,0.214286,0.153846,1,0.984671
5,4762739,17.297765,1.0,1.0,0.5,-25596.02,2.0,-60.55,1.000000,0.250000,0.214286,0.153846,1,0.997027
6,4763087,7957.421355,1.0,1.0,1.0,-25596.02,0.0,482.00,1.000000,0.000000,0.307692,0.153846,1,0.996781
7,4763087,1.487690,0.0,1.0,0.0,-25596.02,22.0,-23.00,1.000000,0.000000,0.307692,0.153846,1,0.997836
8,4763087,5.132853,0.0,0.0,0.0,-25596.02,28.0,47.00,1.000000,0.000000,0.307692,0.153846,1,0.997673
9,4764094,7944.888869,-1.0,-1.0,0.0,-22022.52,4.0,185.45,1.000000,0.296296,0.307692,0.153846,1,0.997498


In [48]:
import numpy as np


def precision_recall_at_k(df: pd.DataFrame, score_col: str, k: int = 3):
    """
    Compute average Precision@k and Recall@k over all orderID groups.

    Args:
      df        : DataFrame containing columns ['orderID', score_col, 'label']
      score_col : name of the column with model scores
      k         : how many top items to consider per group

    Returns:
      (precision, recall) : tuple of floats
    """
    precisions = []
    recalls = []

    # loop over each query (orderID)
    for order_id, group in df.groupby("orderID"):
        # take the top-k by score
        topk = group.nlargest(k, score_col)

        # how many of those are true positives?
        num_true = topk["label"].sum()
        # how many actual positives existed in this group?
        total_true = group["label"].sum()

        precisions.append(num_true / k)
        recalls.append(num_true / total_true if total_true > 0 else 0.0)

    # average across all queries
    return np.mean(precisions), np.mean(recalls)


pt_p, pt_r = precision_recall_at_k(train_pdf, "pt_score", k=3)

In [49]:
print(f"Precision@3: {pt_p:.4f}, Recall@3: {pt_r:.4f}")

Precision@3: 0.9855, Recall@3: 0.9870


In [50]:
subject_df = pl.read_csv('subject_df.csv')
property_df = pl.read_csv('property_df.csv')